# 統計でみる都道府県のすがたを日本地図で可視化

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install
%%sh
pip install jupyter_dash &> /dev/null
pip install -U plotly &> /dev/null
!pip install geopandas &> /dev/null

  Using cached plotly-5.3.1-py2.py3-none-any.whl (23.9 MB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=627db3f9e2bd33fe37df9b885701c349c7f7545c5afe10d5126defa1793e6207
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
    Uninstalling plotly-4.4.1:
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=59b9d850b48942c59923da49b45a2ca207baa753febb8e7b582dc9e184af3a72
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=ac796959249521b5c543930f4f9d692eb25d62227b72fc4674b02449e85afb0b
  Stored in directory: /root/.cache/pip/wheels/85/5d/

In [ ]:
# import
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import ALL, MATCH, Input, Output, State

import requests
from io import BytesIO

import json
import matplotlib.pyplot as plt
import geopandas as gpd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# options
pd.set_option('display.max_columns', 500)

In [ ]:
# load data
df_all = pd.read_csv('/content/drive/MyDrive/codes/prefecture_stat/data/all_data.csv')

In [ ]:
df_all.head()

,都道府県,総人口 (万人),総人口(男) (万人),総人口(女) (万人),外国人人口（人口10万人当たり） (人),全国総人口に占める人口割合 (％),総面積１k㎡当たり人口密度 (人),可住地面積１k㎡当たり人口密度 (人),昼夜間人口比率 (％),人口集中地区人口比率（対総人口） (％),15歳未満人口割合（対総人口） (％),65歳以上人口割合（対総人口） (％),15～64歳人口割合（対総人口） (％),年少人口指数（15歳未満人口／15～64歳人口×100） (－),老年人口指数（65歳以上人口／15～64歳人口×100） (－),従属人口指数((15歳未満人口＋65歳以上人口)/15～64歳人口×100) (－),人口増減率（（総人口－前年総人口）／前年総人口） (％),自然増減率（（出生数－死亡数）／総人口） (％),粗出生率（人口千人当たり） (－),合計特殊出生率 (－),粗死亡率（人口千人当たり） (－),年齢調整死亡率(男)（人口千人当たり） (－),年齢調整死亡率(女)（人口千人当たり） (－),年齢別死亡率(０～４歳)（人口千人当たり） (－),年齢別死亡率(65歳以上)（人口千人当たり） (－),転入超過率(外国人含む)（（転入者数(外国人含む)－転出者数(外国人含む)）／総人口） (％),転入率(外国人含む)(転入者数(外国人含む)/総人口) (％),転出率(外国人含む)(転出者数(外国人含む)/総人口) (％),流入人口比率（対総人口） (％),流出人口比率（対総人口） (％),一般世帯数 (万世帯),全国一般世帯に占める一般世帯割合 (％),一般世帯の平均人員 (人),核家族世帯の割合（対一般世帯数） (％),単独世帯の割合（対一般世帯数） (％),65歳以上の世帯員のいる世帯割合（対一般世帯数） (％),高齢夫婦のみの世帯割合（対一般世帯数） (％),高齢単身世帯の割合（対一般世帯数） (％),共働き世帯割合（対一般世帯数） (％),婚姻率（人口千人当たり） (－),離婚率（人口千人当たり） (－),総面積(北方地域及び竹島を含む) (100k㎡),面積割合(北方地域及び竹島を除く)（対全国総面積） (％),森林面積割合(北方地域及び竹島を除く)（対総面積） (％),自然公園面積割合(北方地域及び竹島を除く)（対総面積） (％),可住地面積割合(北方地域及び竹島を除く)（対総面積） (％),年平均気温 (ﾟC),最高気温（日最高気温の月平均の最高値） (ﾟC),最低気温（日最低気温の月平均の最低値） (ﾟC),年平均相対湿度 (％),日照時間（年間） (時間),降水量（年間） (mm),快晴日数（年間） (日),降水日数（年間） (日),雪日数（年間） (日),１人当たり県民所得（平成23年基準） (千円),県内総生産額対前年増加率（平成23年基準） (％),県民所得対前年増加率（平成23年基準） (％),県民総所得(名目)対前年増加率（平成23年基準） (％),第２次産業事業所数構成比（対事業所数） (％),第３次産業事業所数構成比（対事業所数） (％),従業者１～４人の事業所割合（対民営事業所数） (％),従業者100～299人の事業所割合（対民営事業所数） (％),従業者300人以上の事業所割合（対民営事業所数） (％),第２次産業従業者数（１事業所当たり） (人),第３次産業従業者数（１事業所当たり） (人),従業者１～４人の事業所の従業者割合（対民営事業所従業者数） (％),従業者100～299人の事業所の従業者割合（対民営事業所従業者数） (％),従業者300人以上の事業所の従業者割合（対民営事業所従業者数） (％),就業者１人当たり農業産出額(販売農家) (万円),耕地面積比率（耕地面積／総面積） (％),土地生産性（耕地面積１ヘクタール当たり） (万円),耕地面積（農家１戸当たり） (㎡),製造品出荷額等（従業者１人当たり） (万円),製造品出荷額等（１事業所当たり） (百万円),商業年間商品販売額(卸売業＋小売業)（従業者１人当たり） (万円),商業年間商品販売額(卸売業＋小売業)（１事業所当たり） (百万円),国内銀行預金残高（人口１人当たり） (万円),郵便貯金残高（人口１人当たり） (万円),消費者物価地域差指数（総合） (－),消費者物価地域差指数（家賃を除く総合） (－),消費者物価地域差指数（食料） (－),消費者物価地域差指数（住居） (－),消費者物価地域差指数（光熱・水道） (－),消費者物価地域差指数（家具・家事用品） (－),消費者物価地域差指数（被服及び履物） (－),消費者物価地域差指数（保健医療） (－),消費者物価地域差指数（交通・通信） (－),消費者物価地域差指数（教育） (－),消費者物価地域差指数（教養娯楽） (－),消費者物価地域差指数（諸雑費） (－),標準価格対前年平均変動率(住宅地) (％),財政力指数(都道府県財政) (－),実質収支比率(都道府県財政) (％),地方債現在高の割合（対歳出決算総額）(都道府県財政) (％),経常収支比率(都道府県財政) (％),自主財源の割合（対歳出決算総額）(都道府県財政) (％),一般財源の割合（対歳出決算総額）(都道府県財政) (％),投資的経費の割合（対歳出決算総額）(都道府県財政) (％),地方税割合（対歳入決算総額）(都道府県財政) (％),地方交付税割合（対歳入決算総額）(都道府県財政) (％),国庫支出金割合（対歳入決算総額）(都道府県財政) (％),住民税（人口１人当たり）(都道府県・市町村財政合計) (千円),固定資産税（人口１人当たり）(都道府県・市町村財政合計) (千円),課税対象所得（納税義務者１人当たり） (千円),民生費割合（対歳出決算総額）(都道府県財政) (％),社会福祉費割合（対歳出決算総額)(都道府県財政) (％),老人福祉費割合（対歳出決算総額）(都道府県財政) (％),児童福祉費割合（対歳出決算総額）(都道府県財政) (％),生活保護費割合（対歳出決算総額）(都道府県財政) (％),衛生費割合（対歳出決算総額）(都道府県財政) (％),労働費割合（対歳出決算総額）(都道府県財政) (％),農林水産業費割合（対歳出決算総額）(都道府県財政) (％),商工費割合（対歳出決算総額）(都道府県財政) (％),土木費割合（対歳出決算総額）(都道府県財政) (％),警察費割合（対歳出決算総額）(都道府県財政) (％),消防費割合（対歳出決算総額）(東京都・市町村財政合計) (％),教育費割合（対歳出決算総額）(都道府県財政) (％),災害復旧費割合（対歳出決算総額）(都道府県財政) (％),人件費割合（対歳出決算総額）(都道府県財政) (％),扶助費割合（対歳出決算総額）(都道府県財政) (％),普通建設事業費割合（対歳出決算総額）(都道府県財政) (％),歳出決算総額（人口１人当たり）(都道府県・市町村財政合計) (千円),民生費（人口１人当たり）(都道府県・市町村財政合計) (千円),社会福祉費（人口１人当たり）(都道府県・市町村財政合計) (千円),老人福祉費（65歳以上人口１人当たり）(都道府県・市町村財政合計) (千円),児童福祉費（17歳以下人口１人当たり)(都道府県・市町村財政合計) (千円),生活保護費(被保護実人員１人当たり)(都道府県・市町村財政合計) (千円),衛生費（人口１人当たり）(都道府県・市町村財政合計) (千円),土木費（人口１人当たり）(都道府県・市町村財政合計) (千円),警察費（人口１人当たり）(都道府県財政) (千円),消防費（人口１人当たり）(東京都・市町村財政合計) (千円),教育費（人口１人当たり）(都道府県・市町村財政合計) (千円),社会教育費（人口１人当たり）(都道府県・市町村財政合計) (千円),災害復旧費（人口１人当たり）(都道府県・市町村財政合計) (千円),公立小学校費（児童１人当たり）(都道府県・市町村財政合計) (千円),公立中学校費（生徒１人当たり）(都道府県・市町村財政合計) (千円),公立高等学校費（生徒１人当たり）(都道府県・市町村財政合計) (千円),特別支援学校費（公立）（児童・生徒１人当たり）(都道府県・市町村財政合計) (千円),幼稚園費（児童１人当たり）(都道府県・市町村財政合計) (千円),小学校数（６～11歳人口10万人当たり） (校),中学校数（12～14歳人口10万人当たり） (校),高等学校数（15～17歳人口10万人当たり） (校),幼稚園数（３～５歳人口10万人当たり） (園),保育所等数（０～５歳人口10万人当たり） (所),認定こども園数(０～５歳人口10万人当たり) (園),小学校数(可住地面積100k㎡当たり) (校),中学校数(可住地面積100k㎡当たり) (校),高等学校数(可住地面積100k㎡当たり) (校),小学校教員割合(女)（対小学校教員数） (％),中学校教員割合(女)（対中学校教員数） (％),小学校児童数（小学校教員１人当たり） (人),中学校生徒数（中学校教員１人当たり） (人),高等学校生徒数(高等学校教員１人当たり) (人),幼稚園在園者数(幼稚園教員１人当たり) (人),保育所等在所児数(保育所等保育士１人当たり) (人),公立高等学校生徒比率（対高等学校生徒数） (％),公立幼稚園在園者比率（対幼稚園在園者数） (％),公営保育所等在所児比率（対保育所

In [ ]:
fig = px.bar(df_all, x='都道府県', y='総人口 (万人)')
fig.show()

## 地図の描画

In [ ]:
# 日本の形状データを取得
!wget --no-check-certificate https://github.com/dataofjapan/land/raw/master/japan.geojson

--2021-09-26 09:07:54--  https://github.com/dataofjapan/land/raw/master/japan.geojson
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataofjapan/land/master/japan.geojson [following]
--2021-09-26 09:07:54--  https://raw.githubusercontent.com/dataofjapan/land/master/japan.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13050092 (12M) [text/plain]
Saving to: ‘japan.geojson’

japan.geojson       100%[===================>]  12.45M  --.-KB/s    in 0.1s    

2021-09-26 09:07:54 (107 MB/s) - ‘japan.geojson’ saved [13050092/13050092]



In [ ]:
df2 = gpd.read_file('japan.geojson')
df2.head()

,nam,nam_ja,id,geometry
0,Kyoto Fu,京都府,26,"MULTIPOLYGON (((135.03670 35.53733, 135.03520 ..."
1,Saga Ken,佐賀県,41,"MULTIPOLYGON (((129.86220 33.58753, 129.85770 ..."
2,Kumamoto Ken,熊本県,43,"MULTIPOLYGON (((131.11040 33.18313, 131.11479 ..."
3,Kagawa Ken,香川県,37,"MULTIPOLYGON (((134.33200 34.56313, 134.33510 ..."
4,Aichi Ken,愛知県,23,"MULTIPOLYGON (((136.96091 35.41453, 136.96300 ..."


# jupyterDashによる可視化

In [ ]:
# 選んだ項目をchoropleth_mapboxで描画

GEOJONFILE = "japan.geojson"
df = df_all.dropna()

jsonfile = gpd.read_file(GEOJONFILE)
dfjson = json.loads(jsonfile.to_json())

app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div(
        html.H1('統計でみる都道府県のすがた 2021',
                style={
                    'textAlign': 'center',
                    'color': '#1f77b4'
                })),
    html.Div(html.A(href="https://www.stat.go.jp/data/k-sugata/index.html",
                    target="_blank",
                    children='データ：統計でみる都道府県のすがた(総務省統計局)'),
             style={
                 'textAlign': 'center',
                 'margin': '1% auto'
             }),
    html.Div(children='表示したい項目を選んでください。',
             style={
                 'textAlign': 'center',
                 'margin': '1% auto'
             }),
    html.Div(
        dcc.Dropdown(id='selectplace',
                     options=[{
                         'label': i,
                         'value': i
                     } for i in df.columns.drop('都道府県')],
                     value=df.columns[1],
                     style={
                         'width': 800,
                         'margin': '1% auto'
                     })),
    html.Div(
        dcc.Loading(children=dcc.Graph(id='japanmap',
                                       style={
                                           'width': 1000,
                                           'height': 400,
                                           'margin': '1% auto'
                                       }))),
    html.Div(
        dcc.Loading(children=dcc.Graph(id='hist',
                                       style={
                                           'width': 1100,
                                           'height': 300,
                                           'margin': '1% auto'
                                       })))
])


@app.callback(Output('japanmap', 'figure'), [Input('selectplace', 'value')])
def update_map(selected_value):
    selectdf = df[selected_value]
    fig = px.choropleth_mapbox(selectdf,
                               geojson=dfjson,
                               locations=df['都道府県'],
                               color=selectdf,
                               featureidkey='properties.nam_ja',
                               color_continuous_scale="Viridis",
                               mapbox_style="carto-positron",
                               zoom=3.6,
                               center={
                                   "lat": 36,
                                   "lon": 138
                               },
                               opacity=0.7,
                               labels={"総人口 (万人)": "総人口 (万人)"})

    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    return fig


@app.callback(Output('hist', 'figure'), [Input('selectplace', 'value')])
def draw_graph(selected_value):
    df_sort = df.sort_values(by=selected_value)
    df_sort['順位'] = np.arange(47, 0, -1)
    fig = px.bar(df_sort,
                 x='都道府県',
                 y=selected_value,
                 color=selected_value,
                 color_continuous_scale="Viridis",
                 hover_data=['順位'],
                 title=f'{selected_value}')
    fig.update(layout_coloraxis_showscale=False)
    fig.update_layout(yaxis_title='value')
    return fig


#app.run_server(mode='inline')
app.run_server()


Dash app running on:


<IPython.core.display.Javascript object>